# Customer Segmentation with RFM

## Business Problem

FLO wants to segment its customers and determine marketing strategies based on these segments. For this purpose, customer behaviors will be defined, and groups will be created based on clustering these behavior patterns.

## Dataset Story

The dataset consists of information about customers who made their last purchases in the years 2020-2021 through OmniChannel (both online and offline shopping).

- `master_id`: Unique customer number
- `order_channel`: Channel used for shopping (Android, iOS, Desktop, Mobile, Offline)
- `last_order_channel`: Channel used for the last purchase
- `first_order_date`: Date of the first purchase by the customer
- `last_order_date`: Date of the last purchase by the customer
- `last_order_date_online`: Date of the last online purchase by the customer
- `last_order_date_offline`: Date of the last offline purchase by the customer
- `order_num_total_ever_online`: Total number of purchases made by the customer online
- `order_num_total_ever_offline`: Total number of purchases made by the customer offline
- `customer_value_total_ever_offline`: Total amount spent by the customer in offline purchases
- `customer_value_total_ever_online`: Total amount spent by the customer in online purchases
- `interested_in_categories_12`: List of categories the customer has shopped in the last 12 months

## Tasks

### Task 1: Data Understanding and Preparation

1. Read the `flo_data_20K.csv` data.
2. In the dataset:
   - a. Display the first 10 observations,
   - b. Display variable names,
   - c. Display descriptive statistics,
   - d. Check for missing values,
   - e. Examine variable types.
3. Create new variables for the total number of purchases and spending for each customer, considering that omnichannel customers shop both online and offline.
4. Examine variable types and convert date-related variables to the date type.
5. Explore the distribution of the number of customers, average number of items purchased, and average spending across shopping channels.
6. List the top 10 customers with the highest revenue.
7. List the top 10 customers with the most orders.
8. Modularize the data preprocessing process.

### Task 2: Calculation of RFM Metrics

### Task 3: Calculation of RF and RFM Scores

### Task 4: Definition of RF Scores as Segments

### Task 5: Action Time!

1. Examine the averages of recency, frequency, and monetary for each segment.
2. Using RFM analysis, find customers in the relevant profiles for 2 cases and save their customer IDs to a CSV file.
   - a. FLO is introducing a new women's shoe brand. Special communication will be established with customers interested in this brand. Customers to be targeted are champions and loyal customers, with an average spending of over 250 TL in the women's category. Save the customer IDs to a CSV file named `new_brand_target_customer_ids.csv`.
   - b. A discount of approximately 40% is planned for men and children's products. Customers who were good customers in the past but haven't shopped for a long time, those who are dormant, and new customers will be targeted. Save the customer IDs to a CSV file named `discount_target_customer_ids.csv`.

### Task 6: Functionize the Entire Process


In [1]:
###############################################################
# TASK 1: Data Preparation and Understanding (Data Understanding)
###############################################################

import pandas as pd
import datetime as dt

# Set display options for better visibility
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
pd.set_option('display.float_format', lambda x: '%.2f' % x)
pd.set_option('display.width', 1000)

# 1. Read the "flo_data_20K.csv" data and create a copy of the dataframe.
df_ = pd.read_csv("/kaggle/input/flo-data-20k/flo_data_20k.csv")
df = df_.copy()
df.head()

# 2. In the dataset:
        # a. Display the first 10 observations,
        # b. Display variable names,
        # c. Display dimensions,
        # d. Display descriptive statistics,
        # e. Check for missing values,
        # f. Examine variable types.
df.head(10)
df.columns
df.shape
df.describe().T
df.isnull().sum()
df.info()

# 3. Omnichannel customers shop both online and offline. Create new variables for the total number of purchases and spending for each customer.
df["order_num_total"] = df["order_num_total_ever_online"] + df["order_num_total_ever_offline"]
df["customer_value_total"] = df["customer_value_total_ever_offline"] + df["customer_value_total_ever_online"]

# 4. Examine variable types. Convert date-related variables to the date type.
date_columns = df.columns[df.columns.str.contains("date")]
df[date_columns] = df[date_columns].apply(pd.to_datetime)
df.info()

# 5. Explore the distribution of the number of customers, total number of items purchased, and total spending across shopping channels.
df.groupby("order_channel").agg({"master_id": "count",
                                 "order_num_total": "sum",
                                 "customer_value_total": "sum"})

# 6. List the top 10 customers with the highest revenue.
df.sort_values("customer_value_total", ascending=False)[:10]

# 7. List the top 10 customers with the most orders.
df.sort_values("order_num_total", ascending=False)[:10]

# 8. Modularize the data preprocessing process.
def data_prep(dataframe):
    dataframe["order_num_total"] = dataframe["order_num_total_ever_online"] + dataframe["order_num_total_ever_offline"]
    dataframe["customer_value_total"] = dataframe["customer_value_total_ever_offline"] + dataframe["customer_value_total_ever_online"]
    date_columns = dataframe.columns[dataframe.columns.str.contains("date")]
    dataframe[date_columns] = dataframe[date_columns].apply(pd.to_datetime)
    return df


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 19945 entries, 0 to 19944
Data columns (total 12 columns):
 #   Column                             Non-Null Count  Dtype  
---  ------                             --------------  -----  
 0   master_id                          19945 non-null  object 
 1   order_channel                      19945 non-null  object 
 2   last_order_channel                 19945 non-null  object 
 3   first_order_date                   19945 non-null  object 
 4   last_order_date                    19945 non-null  object 
 5   last_order_date_online             19945 non-null  object 
 6   last_order_date_offline            19945 non-null  object 
 7   order_num_total_ever_online        19945 non-null  float64
 8   order_num_total_ever_offline       19945 non-null  float64
 9   customer_value_total_ever_offline  19945 non-null  float64
 10  customer_value_total_ever_online   19945 non-null  float64
 11  interested_in_categories_12        19945 non-null  obj

In [2]:
###############################################################
# TASK 2: Calculation of RFM Metrics
###############################################################

# Analysis date, two days after the date of the last purchase in the dataset
df["last_order_date"].max()  # 2021-05-30
analysis_date = dt.datetime(2021, 6, 1)

# Create a new RFM dataframe containing customer_id, recency, frequency, and monetary values
rfm = pd.DataFrame()
rfm["customer_id"] = df["master_id"]
rfm["recency"] = (analysis_date - df["last_order_date"]).dt.days.astype(int)
rfm["frequency"] = df["order_num_total"]
rfm["monetary"] = df["customer_value_total"]

rfm.head()



,customer_id,recency,frequency,monetary
0,cc294636-19f0-11eb-8d74-000d3a38a36f,95,5.00,939.37
1,f431bd5a-ab7b-11e9-a2fc-000d3a38a36f,105,21.00,2013.55
2,69b69676-1a40-11ea-941b-000d3a38a36f,186,5.00,585.32
3,1854e56c-491f-11eb-806e-000d3a38a36f,135,2.00,121.97
4,d6ea1074-f1f5-11e9-9346-000d3a38a36f,86,2.00,209.98


In [3]:
###############################################################
# TASK 3: Calculating RF and RFM Scores
###############################################################

# Convert Recency, Frequency, and Monetary metrics to scores between 1-5 using qcut,
# and save these scores as recency_score, frequency_score, and monetary_score
rfm["recency_score"] = pd.qcut(rfm['recency'], 5, labels=[5, 4, 3, 2, 1])
rfm["frequency_score"] = pd.qcut(rfm['frequency'].rank(method="first"), 5, labels=[1, 2, 3, 4, 5])
rfm["monetary_score"] = pd.qcut(rfm['monetary'], 5, labels=[1, 2, 3, 4, 5])

rfm.head()

# Combine recency_score and frequency_score into a single variable and save it as RF_SCORE
rfm["RF_SCORE"] = (rfm['recency_score'].astype(str) + rfm['frequency_score'].astype(str))

# Combine recency_score, frequency_score, and monetary_score into a single variable and save it as RFM_SCORE
rfm["RFM_SCORE"] = (rfm['recency_score'].astype(str) + rfm['frequency_score'].astype(str) + rfm['monetary_score'].astype(str))

rfm.head()


,customer_id,recency,frequency,monetary,recency_score,frequency_score,monetary_score,RF_SCORE,RFM_SCORE
0,cc294636-19f0-11eb-8d74-000d3a38a36f,95,5.00,939.37,3,4,4,34,344
1,f431bd5a-ab7b-11e9-a2fc-000d3a38a36f,105,21.00,2013.55,3,5,5,35,355
2,69b69676-1a40-11ea-941b-000d3a38a36f,186,5.00,585.32,2,4,3,24,243
3,1854e56c-491f-11eb-806e-000d3a38a36f,135,2.00,121.97,3,1,1,31,311
4,d6ea1074-f1f5-11e9-9346-000d3a38a36f,86,2.00,209.98,3,1,1,31,311


In [4]:
###############################################################
# TASK 4: Definition of RF Scores as Segments
###############################################################

# Define segments for the created RFM scores and convert RF_SCORE to segments using the defined seg_map for better interpretation
seg_map = {
    r'[1-2][1-2]': 'hibernating',
    r'[1-2][3-4]': 'at_Risk',
    r'[1-2]5': 'cant_loose',
    r'3[1-2]': 'about_to_sleep',
    r'33': 'need_attention',
    r'[3-4][4-5]': 'loyal_customers',
    r'41': 'promising',
    r'51': 'new_customers',
    r'[4-5][2-3]': 'potential_loyalists',
    r'5[4-5]': 'champions'
}

# Replace RF_SCORE with the corresponding segments using seg_map
rfm['segment'] = rfm['RF_SCORE'].replace(seg_map, regex=True)

rfm.head()


,customer_id,recency,frequency,monetary,recency_score,frequency_score,monetary_score,RF_SCORE,RFM_SCORE,segment
0,cc294636-19f0-11eb-8d74-000d3a38a36f,95,5.00,939.37,3,4,4,34,344,loyal_customers
1,f431bd5a-ab7b-11e9-a2fc-000d3a38a36f,105,21.00,2013.55,3,5,5,35,355,loyal_customers
2,69b69676-1a40-11ea-941b-000d3a38a36f,186,5.00,585.32,2,4,3,24,243,at_Risk
3,1854e56c-491f-11eb-806e-000d3a38a36f,135,2.00,121.97,3,1,1,31,311,about_to_sleep
4,d6ea1074-f1f5-11e9-9346-000d3a38a36f,86,2.00,209.98,3,1,1,31,311,about_to_sleep


In [5]:
###############################################################
# TASK 5: Action Time!
###############################################################

# 1. Examine the averages of recency, frequency, and monetary for each segment.
rfm[["segment", "recency", "frequency", "monetary"]].groupby("segment").agg(["mean", "count"])


recency       frequency       monetary      
                       mean count      mean count     mean count
segment                                                         
about_to_sleep       113.79  1629      2.40  1629   359.01  1629
at_Risk              241.61  3131      4.47  3131   646.61  3131
cant_loose           235.44  1200     10.70  1200  1474.47  1200
champions             17.11  1932      8.93  1932  1406.63  1932
hibernating          247.95  3604      2.39  3604   366.27  3604
loyal_customers       82.59  3361      8.37  3361  1216.82  3361
need_attention       113.83   823      3.73   823   562.14   823
new_customers         17.92   680      2.00   680   339.96   680
potential_loyalists   37.16  2938      3.30  2938   533.18  2938
promising             58.92   647      2.00   647   335.67   647

In [6]:
# 2. Using RFM analysis, find customers in the relevant profiles for 2 cases and save their customer IDs to a CSV file.

## a. FLO is introducing a new women's shoe brand. Special communication will be established with loyal customers and those who shop in the women's category. 
## Customer IDs will be saved to a CSV file named "new_brand_target_customer_ids.csv".

target_segments_customer_ids_a = rfm[rfm["segment"].isin(["champions", "loyal_customers"])]["customer_id"]
customer_ids_a = df[(df["master_id"].isin(target_segments_customer_ids_a)) & (df["interested_in_categories_12"].str.contains("KADIN"))]["master_id"]
customer_ids_a.to_csv("new_brand_target_customer_ids.csv", index=False)
customer_ids_a.shape

## b. A discount of approximately 40% is planned for men and children's products. Customers who were good customers in the past but haven't shopped for a long time,
## and new customers will be targeted. Save the customer IDs to a CSV file named "discount_target_customer_ids.csv".

target_segments_customer_ids_b = rfm[rfm["segment"].isin(["cant_loose", "hibernating", "new_customers"])]["customer_id"]
customer_ids_b = df[(df["master_id"].isin(target_segments_customer_ids_b)) & ((df["interested_in_categories_12"].str.contains("ERKEK")) | (df["interested_in_categories_12"].str.contains("COCUK")))]["master_id"]
customer_ids_b.to_csv("discount_target_customer_ids.csv", index=False)
customer_ids_b.shape


(2771,)

In [7]:
###############################################################
# BONUS
###############################################################

def create_rfm(dataframe):
    # Data Preparation
    dataframe["order_num_total"] = dataframe["order_num_total_ever_online"] + dataframe["order_num_total_ever_offline"]
    dataframe["customer_value_total"] = dataframe["customer_value_total_ever_offline"] + dataframe["customer_value_total_ever_online"]
    date_columns = dataframe.columns[dataframe.columns.str.contains("date")]
    dataframe[date_columns] = dataframe[date_columns].apply(pd.to_datetime)

    # Calculation of RFM Metrics
    dataframe["last_order_date"].max()  # 2021-05-30
    analysis_date = dt.datetime(2021, 6, 1)
    rfm = pd.DataFrame()
    rfm["customer_id"] = dataframe["master_id"]
    rfm["recency"] = (analysis_date - dataframe["last_order_date"]).dt.days
    rfm["frequency"] = dataframe["order_num_total"]
    rfm["monetary"] = dataframe["customer_value_total"]

    # Calculation of RF and RFM Scores
    rfm["recency_score"] = pd.qcut(rfm['recency'], 5, labels=[5, 4, 3, 2, 1])
    rfm["frequency_score"] = pd.qcut(rfm['frequency'].rank(method="first"), 5, labels=[1, 2, 3, 4, 5])
    rfm["monetary_score"] = pd.qcut(rfm['monetary'], 5, labels=[1, 2, 3, 4, 5])
    rfm["RF_SCORE"] = (rfm['recency_score'].astype(str) + rfm['frequency_score'].astype(str))
    rfm["RFM_SCORE"] = (rfm['recency_score'].astype(str) + rfm['frequency_score'].astype(str) + rfm['monetary_score'].astype(str))

    # Naming of Segments
    seg_map = {
        r'[1-2][1-2]': 'hibernating',
        r'[1-2][3-4]': 'at_Risk',
        r'[1-2]5': 'cant_loose',
        r'3[1-2]': 'about_to_sleep',
        r'33': 'need_attention',
        r'[3-4][4-5]': 'loyal_customers',
        r'41': 'promising',
        r'51': 'new_customers',
        r'[4-5][2-3]': 'potential_loyalists',
        r'5[4-5]': 'champions'
    }
    rfm['segment'] = rfm['RF_SCORE'].replace(seg_map, regex=True)

    return rfm[["customer_id", "recency", "frequency", "monetary", "RF_SCORE", "RFM_SCORE", "segment"]]


rfm_df = create_rfm(df)
rfm_df.head()

,customer_id,recency,frequency,monetary,RF_SCORE,RFM_SCORE,segment
0,cc294636-19f0-11eb-8d74-000d3a38a36f,95,5.00,939.37,34,344,loyal_customers
1,f431bd5a-ab7b-11e9-a2fc-000d3a38a36f,105,21.00,2013.55,35,355,loyal_customers
2,69b69676-1a40-11ea-941b-000d3a38a36f,186,5.00,585.32,24,243,at_Risk
3,1854e56c-491f-11eb-806e-000d3a38a36f,135,2.00,121.97,31,311,about_to_sleep
4,d6ea1074-f1f5-11e9-9346-000d3a38a36f,86,2.00,209.98,31,311,about_to_sleep
